In [14]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 

In [2]:
results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)

In [5]:
cities_df = pd.read_csv("../../../european-city-data/archive/city_abstracts_embeddings.csv")
cities = list(cities_df['city'])

In [24]:
import spacy
from geopy.geocoders import Nominatim
nlp = spacy.load("en_core_web_sm")
import geonamescache

gc = geonamescache.GeonamesCache()

# Download the required NLTK models (only needed once)
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

def find_all_cities(text):
    # Initialize the geolocator
    geolocator = Nominatim(user_agent="city_finder")

    # Process the passage with spaCy to perform NER
    doc = nlp(passage)

    cities = []

    # Iterate through the recognized entities
    for entity in doc.ents:
        if entity.label_ == "GPE" and entity not in cities:  # GPE (Geopolitical Entity) often includes cities
            # gc.search_cities('NAME', case_sensitive=True, contains_search=True)
            # location = geolocator.geocode(entity.text)
            # if location and location.raw.get('class') == 'place' and entity.text not in cities:
            # print(entity)
            cities.append(entity)

    return cities


# Example usage
passage = "I have visited New York, Greece, Paris, and London. Next year, I plan to go to Tokyo and maybe Sydney."
cities = find_all_cities(passage)
print(cities)

[New York, Greece, Paris, London, Tokyo, Sydney]


In [27]:
gc.search_cities('USA', case_sensitive=True, contains_search=False)

[{'geonameid': 4461574,
  'name': 'Concord',
  'latitude': 35.40888,
  'longitude': -80.58158,
  'countrycode': 'US',
  'population': 87696,
  'timezone': 'America/New_York',
  'admin1code': 'NC',
  'alternatenames': ['Concord',
   'Konkord',
   'USA',
   'kankrd  karwlynay shmaly',
   'konkodo',
   'kwnkwrd',
   'Конкорд',
   'كونكورد',
   'کانکرد، کارولینای شمالی',
   'کونکورڈ، شمالی کیرولائنا',
   'コンコード']},
 {'geonameid': 4956184,
  'name': 'Worcester',
  'latitude': 42.26259,
  'longitude': -71.80229,
  'countrycode': 'US',
  'population': 206518,
  'timezone': 'America/New_York',
  'admin1code': 'MA',
  'alternatenames': ['Grafton Gore',
   'ORH',
   'Ouorster',
   'Pakachoog',
   'Uorsestehr',
   'Ustur',
   'Vigornia',
   'Vustehr',
   'Vuster',
   'Vusteris',
   'Worcester',
   'Worcester i USA',
   'useuteo',
   'usuta',
   'wrsstr',
   'wu si te',
   'wwstr',
   'wwstr  masachwst',
   'Ουόρστερ',
   'Вустер',
   'Вустэр',
   'Уорсестэр',
   'Устър',
   'ווסטר',
   'ورسستر',


In [11]:
def extract_city(text):
    # Define the regex pattern to find the city name right after "I recommend"
    pattern = r'I recommend\s+(?:visiting\s+)?(?:the city of\s+)?(\w+)'
    match = re.search(pattern, text)
    
    # If a match is found, return the captured group
    if match:
        return match.group(1)
    else:
        return None
    
def extract_first_list_item(text):
    # Define the regex pattern to find the first item in a numbered list
    pattern = r'\b1\.\s+([^0-9]+)(?=\s*\b[2-9]\.)(\w+)'
    
    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)
    
    # If a match is found, return the captured group
    if match:
        first_item = match.group(1).strip()
        cleaned_item = re.sub(r'[^a-zA-Z\s]', '', first_item)
        cleaned_item = cleaned_item.strip().split(" ")[0]
        # return cleaned_item
        if cleaned_item in cities:
            return cleaned_item
    else:
        return None
    
def find_first_city(paragraph):
    # Create a regex pattern to match any city in the list
    city_pattern = r'\b(' + '|'.join(re.escape(city) for city in cities) + r')\b'
    
    # Search for the pattern in the paragraph
    match = re.search(city_pattern, paragraph, re.IGNORECASE)
    
    # If a match is found, return the matched city name
    if match:
        return match.group(1)
    else:
        return None

results_dict = []

for model in folders:
    if ".csv" in model: 
        continue 

    for prompt in os.listdir(os.path.join(results_dir, model)):
        
        with open(os.path.join(results_dir, model, prompt, "response.txt")) as f:
            response = f.read()
        
        with open(os.path.join(results_dir, model, prompt, "response_sustainable.txt")) as f:
            response_sustainable = f.read()

        rec_cities = find_all_cities(response)

        # if city is None or city not in cities:
        #     # print(model, prompt)
        #     city = extract_first_list_item(response)

        #     if city is None or city not in cities:
        #         city = find_first_city(response)

        rec_cities_sustainable = find_all_cities(response_sustainable)

        # if city_sustainable is None or city_sustainable not in cities:
        #     city_sustainable = extract_first_list_item(response_sustainable)

        #     if city_sustainable is None or city_sustainable not in cities:
        #         city_sustainable = find_first_city(response_sustainable)

        results_dict.append({
            'model': model, 
            'prompt_id': prompt,
            'rec_city': rec_cities, 
            'response': response,
            'rec_city_sustainable': rec_cities_sustainable,
            'response_sustainable': response_sustainable
        })
        
results_df = pd.DataFrame(results_dict)

KeyboardInterrupt: 

In [ ]:
results_df.head(5)

,model,prompt_id,rec_city,response,rec_city_sustainable,response_sustainable
0,llama3point1-instruct,prompt_17_gemini-ui,"[Kaunas, Kaunas, Kaunas, Kaunas, Kaunas]",I recommend Kaunas in Lithuania for your expe...,"[Vitoria-Gasteiz, Vitoria-Gasteiz]",I recommend Vitoria-Gasteiz because it is a c...
1,llama3point1-instruct,prompt_42_gpt-4o-mini,"[Amsterdam, Amsterdam, Amsterdam, Amsterdam, A...","I recommend Amsterdam, Netherlands. Amsterdam...","[Nalchik, Varna, Varna, Kahramanmaras, Kahrama...","I recommend Nalchik, Russia because of its be..."
2,llama3point1-instruct,prompt_0_gpt-4o-mini,"[Kaunas, Kaunas, Kaunas, Thessaloniki, Thessal...","I recommend Kaunas, Lithuania. I recommend Ka...","[Kaunas, Kaunas, Kaunas, Kaunas, Kaunas, Kauna...","I recommend Kaunas, Lithuania because it is a..."
3,llama3point1-instruct,prompt_32_gpt-4o-mini,"[Kaunas, Kaunas, Munich, Munich, Varna, Arkhan...","I recommend Kaunas, Lithuania because it offe...","[Munich, Munich, Munich, Munich, Munich]","I recommend Munich, Germany. I recommend Muni..."
4,llama3point1-instruct,prompt_7_gemini-ui,"[Sibiu, Sibiu, Sibiu, Sibiu, Sibiu, Vitoria-Ga...","I recommend Sibiu, Romania in October. I reco...","[Vitoria-Gasteiz, Vitoria-Gasteiz, Gaziantep, ...","I recommend Vitoria-Gasteiz, Spain, because i..."


In [27]:
results_df.to_csv(os.path.join(results_dir, "recommended_cities.csv"))

In [44]:
"Venice" in cities

False